In [9]:

import os
from collections import defaultdict

def get_turn_ids(path_qrel):
    with open(path_qrel) as f:
        lines = f.readlines()
    turn_ids = []    
    for line in lines:
        turn_ids.append(line.split(' ')[0].strip())

    
    turn_ids= list(set(turn_ids))
    print(len(turn_ids))
    return turn_ids


def get_top_passages_per_run(ranking_models_output_path_dir, pruned_turns):
    run_names = list(os.listdir(ranking_models_output_path_dir))

    assert len(run_names)==28

    run_top10_passages = defaultdict(list)

    for run_name in run_names:
        with open(ranking_models_output_path_dir+run_name, 'r') as f:
            lines = f.readlines()
        
        for line in lines:
            turn_id, _, passage_id, rank, _, _ =line.split('\t')
            if turn_id in pruned_turns:
                query_passage_pair_id = turn_id + '****' + passage_id
                if int(rank)<=10:
                    run_top10_passages[run_name].append(query_passage_pair_id)

    return run_top10_passages


def create_new_subset_pool(new_pool_path, nist_pool_path, turns_to_remove):
    
    new_lines = []

    with open(new_pool_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        turn_id, _, passage_id, score = line.split('\t')
        qp_pair_id = turn_id+'****'+passage_id

        if qp_pair_id in turns_to_remove:
            new_lines.append(line)

    with open(nist_pool_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        turn_id, _, passage_id, score = line.split(' ')
        qp_pair_id = turn_id+'****'+passage_id

        if not(qp_pair_id in turns_to_remove):
            new_lines.append(line.replace(' ', '\t'))

    print(len(new_lines))
    print(len(lines))

    return new_lines


def get_pool_of_run_x(run_name_x, dict_run_top10_passages):

    pool_rest_runs = []
    pool_run_x = []
    
    for run_name in dict_run_top10_passages:
        if run_name!=run_name_x:
            pool_rest_runs+= dict_run_top10_passages[run_name]

    pool_run_x = [sample for sample in dict_run_top10_passages[run_name_x] if not (sample in pool_rest_runs)]

    print(len(pool_run_x))
    
    return pool_run_x

In [5]:
pruned_turns = get_turn_ids('data/human_qrels')
ranking_models_output_path_dir = 'trec_format/'
dict_run_top10_passages = get_top_passages_per_run(ranking_models_output_path_dir, pruned_turns)

133


In [ ]:
ranking_models_output_path_dir = 'trec_format/'
dict_run_top10_passages = get_top_passages_per_run(ranking_models_output_path_dir, pruned_turns)

nist_pool_path  = 'data/human_qrels'
output_dir_path = 'tmp_pool_remove_one_run_llama_zero/'
new_pool_path   = 'llama-zero-shot-models/ikat.Meta-Llama-3-8B-Instruct'


for run_name in dict_run_top10_passages:
    output_run_path = output_dir_path + run_name
    turns_to_remove = get_pool_of_run_x(run_name, dict_run_top10_passages)
    
    res = create_new_subset_pool(new_pool_path, nist_pool_path, turns_to_remove)
    with open(output_run_path, 'w') as f:
        f.writelines(res)
    print(run_name)
    